In [14]:
import pandas as pd
from databaker.framework import *
from tutorialResources.scraper import Scraper

scraper = Scraper("https://www.fake-website.com/example1")

tabs = scraper.distribution.as_databaker()


# Apply

The databaker apply keyword is a means to pass a callable (a function, a class or a lambda function) into a HDim constructor.

There are two drivers for this:
    
- Simplification of transforms
- Performance - if you modify something before you pivot it, it's **very** efficient compared to post processing.

The following is just the unmodified transform.

In [15]:

tidied_sheets = []
for tab in tabs:       

    # define a selection of cells as the observations
    observations = tab.excel_ref('C5').expand(DOWN).expand(RIGHT).is_not_blank()
    
    # define other selections of cells to be our dimensions
    assets = tab.excel_ref('C3').expand(RIGHT).is_not_blank()
    names = tab.excel_ref('B5').expand(DOWN).is_not_blank()
    group = tab.excel_ref('A5').expand(DOWN).is_not_blank()

    #define the relationships of the cells selected as dimensions (relative to the observations)
    dimensions = [
              HDim(assets, "Assets", DIRECTLY, ABOVE), 
              HDim(names, "Name", DIRECTLY, LEFT), 
              HDim(group, "Group", CLOSEST, ABOVE) 
                 ]
    
    # Now we process these relationship for this tab (this code rarely changes)
    tidy_sheet = ConversionSegment(tab, dimensions, observations) # < --- processing
    #savepreviewhtml(tidy_sheet)
    
    tidied_sheets.append(tidy_sheet.topandas()) # <-- adding result of processing this tab to our list
    
output = pd.concat(tidied_sheets)
output[:10]

,OBS,Assets,Name,Group
0,1.0,Houses,John,Beatles
1,6.0,Cars,John,Beatles
2,1.0,Businesses,John,Beatles
3,2.0,Houses,Paul,Beatles
4,4.0,Cars,Paul,Beatles
5,6.0,Businesses,Paul,Beatles
6,3.0,Houses,George,Beatles
7,3.0,Cars,George,Beatles
8,2.0,Businesses,George,Beatles
9,8.0,Houses,Ringo,Beatles


# Example 1:
    
Let's add a "The" prefix to the group names:

Note - the only change is the "group" HDim

In [10]:
class is_one_of:
    """A filter that will return true if the cell value is one of a defined list of values"""

    def __init__(self, our_chosen_values):
        self.our_chosen_values = our_chosen_values

    def __call__(self, xy_cell):
        if xy_cell.value in self.our_chosen_values:
            return True
        return False

import pandas as pd
from databaker.framework import *
from tutorialResources.scraper import Scraper

scraper = Scraper("https://www.fake-website.com/example1")

tabs = scraper.distribution.as_databaker()

for tab in tabs:       

    # define a selection of cells as the observations
    observations = tab.filter(is_one_of([1.0, 2.0]))
    
    print(observations)

{<E5 1.0>, <C5 1.0>}


# Example 2:
    
Let's pass in a more complicated function that url'ify the beatles but ignore the rolling stones.
    
Note - the only change is the "name" HDim

In [17]:
def beatles_as_urls(value):
    d = {
        "John": "beatles/john/singer",
        "Paul": "beatles/paul/bass",
        "Ringo": "beatles/ringo/drums",
        "George": "beatles/geaoge/guitar"
    }
    return d.get(value, value)

tidied_sheets = []
for tab in tabs:       

    # define a selection of cells as the observations
    observations = tab.excel_ref('C5').expand(DOWN).expand(RIGHT).is_not_blank()
    
    # define other selections of cells to be our dimensions
    assets = tab.excel_ref('C3').expand(RIGHT).is_not_blank()
    names = tab.excel_ref('B5').expand(DOWN).is_not_blank()
    group = tab.excel_ref('A5').expand(DOWN).is_not_blank()

    #define the relationships of the cells selected as dimensions (relative to the observations)
    dimensions = [
              HDim(assets, "Assets", DIRECTLY, ABOVE), 
              HDim(names, "Name", DIRECTLY, LEFT, apply=beatles_as_urls), 
              HDim(group, "Group", CLOSEST, ABOVE) 
                 ]
    
    # Now we process these relationship for this tab (this code rarely changes)
    tidy_sheet = ConversionSegment(tab, dimensions, observations) # < --- processing
    #savepreviewhtml(tidy_sheet)
    
    tidied_sheets.append(tidy_sheet.topandas()) # <-- adding result of processing this tab to our list
    
output = pd.concat(tidied_sheets)
output[:10]

,OBS,Assets,Name,Group
0,1.0,Houses,beatles/john/singer,Beatles
1,6.0,Cars,beatles/john/singer,Beatles
2,1.0,Businesses,beatles/john/singer,Beatles
3,2.0,Houses,beatles/paul/bass,Beatles
4,4.0,Cars,beatles/paul/bass,Beatles
5,6.0,Businesses,beatles/paul/bass,Beatles
6,3.0,Houses,beatles/geaoge/guitar,Beatles
7,3.0,Cars,beatles/geaoge/guitar,Beatles
8,2.0,Businesses,beatles/geaoge/guitar,Beatles
9,8.0,Houses,beatles/ringo/drums,Beatles


# Example 3

You can pass in a sequence if callables via a tuple if you want to.

In this case we're going to pass in two lambda functions:

- Add a prefix of "lots of"
- Replace the term "lots" with "many types" but _only_ for houses

For this one look at the Assets HDim constructor.

_Note - pretty ugly but this is just a quick example_.

In [19]:
tidied_sheets = []
for tab in tabs:       

    # define a selection of cells as the observations
    observations = tab.excel_ref('C5').expand(DOWN).expand(RIGHT).is_not_blank()
    
    # define other selections of cells to be our dimensions
    assets = tab.excel_ref('C3').expand(RIGHT).is_not_blank()
    names = tab.excel_ref('B5').expand(DOWN).is_not_blank()
    group = tab.excel_ref('A5').expand(DOWN).is_not_blank()

    #define the relationships of the cells selected as dimensions (relative to the observations)
    dimensions = [
              HDim(assets, "Assets", DIRECTLY, ABOVE, apply=(lambda x: f'lots of {x}',lambda x: x.replace("lots", "many types") if "Houses" in x else x)), 
              HDim(names, "Name", DIRECTLY, LEFT), 
              HDim(group, "Group", CLOSEST, ABOVE) 
                 ]
    
    # Now we process these relationship for this tab (this code rarely changes)
    tidy_sheet = ConversionSegment(tab, dimensions, observations) # < --- processing
    #savepreviewhtml(tidy_sheet)
    
    tidied_sheets.append(tidy_sheet.topandas()) # <-- adding result of processing this tab to our list
    
output = pd.concat(tidied_sheets)
output[:10]

,OBS,Assets,Name,Group
0,1.0,many types of Houses,John,Beatles
1,6.0,lots of Cars,John,Beatles
2,1.0,lots of Businesses,John,Beatles
3,2.0,many types of Houses,Paul,Beatles
4,4.0,lots of Cars,Paul,Beatles
5,6.0,lots of Businesses,Paul,Beatles
6,3.0,many types of Houses,George,Beatles
7,3.0,lots of Cars,George,Beatles
8,2.0,lots of Businesses,George,Beatles
9,8.0,many types of Houses,Ringo,Beatles
